## Harga Harta Kediaman

single

In [9]:
import pandas as pd
from io import BytesIO
import requests
import numpy as np

# Sample URLs
urls = [
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/WP KL/Jadual Harga dan Sewa WPKL LPH 2022.xlsx",
    # "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/WP LABUAN/Jadual Harga dan Sewa  Labuan LPH 2022.xlsx",
    # Add more URLs here
]

# Create an empty list to store DataFrames
dfs = []

# Specify the sheet names you want to read
target_sheets = ['.3']  # Sheets ending with '.1' or '.3'

# Iterate through each URL
for url in urls:
    # Get the content of the file from the URL
    response = requests.get(url)
    
    # Read sheets that match the criteria
    xls = pd.ExcelFile(BytesIO(response.content))
    sheet_names = [sheet for sheet in xls.sheet_names if sheet.endswith(tuple(target_sheets))]
    
    # Extract relevant information from the URL
    url_parts = url.split('/')
    url_file = f"{url_parts[-3]}/{url_parts[-2]}"
    
    # Iterate through each sheet
    for sheet in sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet, skiprows=5)#,
        df = df.iloc[1:,:4]
        df.columns = ['district_mukim','avg_floorsm','rental_2021','rental_2022']
        
        # Drop rows with all NaN values
        df = df.dropna(how='all')
        
        # Check if df[1] is NaN and use df[0] value for a new column 'location'
        df['location'] = np.where(pd.isna(df[df.columns[1]]), df[df.columns[0]], df[df.columns[1]])
        df['location'] = df['location'].astype(str)
                
        # Identify strings containing numbers and replace with NaN
        pattern = r'\d'  # Regular expression to match any digit
        df['location'] = df['location'].where(~df['location'].str.contains(pattern), np.nan)

        # Forward-fill the 'location' column
        df['location'] = df['location'].fillna(method='ffill')
        
        # Create a new column 'types' at rows where df[0] is in uppercase
        df['types'] = np.where(df[df.columns[0]].str.isupper(), df[df.columns[0]], np.nan)
        
        # Forward-fill the 'types' column until a different value is encountered
        df['types'] = df['types'].fillna(method='ffill')

        # Remove rows where 'district_mukim' is equal to 'location'
        df = df[df[df.columns[0]] != df[df.columns[4]]]
                
        # Add a new column 'filename' with the extracted information
        df['source'] = url_file
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
final_result_df = pd.concat(dfs, ignore_index=True)

# Display the final result DataFrame
final_result_df.head(30)
df

,district_mukim,avg_floorsm,rental_2021,rental_2022,location,types,source
5,Taman Maluri,75.00,"1,200 - 1,600","1,200 - 1,600",Mukim Ampang,SINGLE STOREY TERRACE,2022/WP KL
8,Bangsar Park,129.00,"2,300 - 2,500","2,300 - 2,500",Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
9,Cheras Heights Condominium (Bukit Cheras),200.00,NaN,1800,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
10,Lucky Garden,110.00,"2,800 - 3,000","2,800 - 3,000",Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
11,Taman Desa Bakti,84.17,1200,1200,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
...,...,...,...,...,...,...,...
489,Taman Sri Hartamas,137.00,"1,800 - 2,200","1,800 - 2,200",Mukim Kuala Lumpur,TOWNHOUSE,2022/WP KL
492,Sunway SPK 3 Harmoni,217.00,"3,000 - 3,300","3,000 - 3,300",Mukim Batu,TOWNHOUSE,2022/WP KL
493,Villa Mas Townhouse,192.00,"1,500 - 1,600","1,500 - 1,600",Mukim Batu,TOWNHOUSE,2022/WP KL
496,Sunway Montana,250.00,4000,4000,Mukim Setapak,TOWNHOUSE,2022/WP KL


Bulk

In [ ]:
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/WP KL/Jadual Harga dan Sewa WPKL LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/WP LABUAN/Jadual Harga dan Sewa  Labuan LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/WP PUTRAJAYA/Jadual Harga dan Sewa Putrajaya LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/KEDAH/Jadual Harga dan Sewa Kedah LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/PERLIS/Jadual Harga dan Sewa Perlis LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/PULAU PINANG/Jadual Harga dan Sewa Pulau Pinang LPH 2022.xls
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/SELANGOR/Jadual Harga dan Sewa Selangor LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/PAHANG/Jadual Harga dan Sewa Pahang LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/MELAKA/Jadual Harga dan Sewa Melaka LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/NEGERI SEMBILAN/Jadual Harga dan Sewa N. Sembilan LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/PERAK/Jadual Harga dan Sewa Perak LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/TERENGGANU/Jadual Harga dan Sewa Terengganu LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/KELANTAN/Jadual Harga dan Sewa Kelantan LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/JOHOR/Jadual Harga dan Sewa Johor LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/SABAH/Jadual Harga dan Sewa Sabah LPH 2022.xlsx",
"https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/SARAWAK/Jadual Harga dan Sewa Sarawak LPH 2022.xlsx",

In [10]:
import pandas as pd
from io import BytesIO
import requests
import numpy as np

# Sample URLs
urls = [
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/WP KL/Jadual Harga dan Sewa WPKL LPH 2022.xlsx",
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/WP LABUAN/Jadual Harga dan Sewa  Labuan LPH 2022.xlsx",
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/WP PUTRAJAYA/Jadual Harga dan Sewa Putrajaya LPH 2022.xlsx",
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/KEDAH/Jadual Harga dan Sewa Kedah LPH 2022.xlsx",
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/PERLIS/Jadual Harga dan Sewa Perlis LPH 2022.xlsx",
    # "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/PULAU PINANG/Jadual Harga dan Sewa Pulau Pinang LPH 2022.xlsx",
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/SELANGOR/Jadual Harga dan Sewa Selangor LPH 2022.xlsx",
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/PAHANG/Jadual Harga dan Sewa Pahang LPH 2022.xlsx",
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/MELAKA/Jadual Harga dan Sewa Melaka LPH 2022.xlsx",
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/NEGERI SEMBILAN/Jadual Harga dan Sewa N. Sembilan LPH 2022.xlsx",
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/PERAK/Jadual Harga dan Sewa Perak LPH 2022.xlsx",
    # "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/TERENGGANU/Jadual Harga dan Sewa Terengganu LPH 2022.xlsx",
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/KELANTAN/Jadual Harga dan Sewa Kelantan LPH 2022.xlsx",
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/JOHOR/Jadual Harga dan Sewa Johor LPH 2022.xlsx",
    # "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/SABAH/Jadual Harga dan Sewa Sabah LPH 2022.xlsx",
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/SARAWAK/Jadual Harga dan Sewa Sarawak LPH 2022.xlsx"
    ]

# Create an empty list to store DataFrames
dfs = []

# Specify the sheet names you want to read
target_sheets = ['.3']  # Sheets ending with '.1' or '.3'

# Iterate through each URL
for url in urls:
    # Get the content of the file from the URL
    response = requests.get(url)
    
    # Read sheets that match the criteria
    xls = pd.ExcelFile(BytesIO(response.content))
    sheet_names = [sheet for sheet in xls.sheet_names if sheet.endswith(tuple(target_sheets))]
    
    # Extract relevant information from the URL
    url_parts = url.split('/')
    url_file = f"{url_parts[-3]}/{url_parts[-2]}"
    
    # Iterate through each sheet
    for sheet in sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet, skiprows=5)#,
        df = df.iloc[1:,:4]
        df.columns = ['district_mukim','avg_floorsm','rental_2021','rental_2022']
        
        # Drop rows with all NaN values
        df = df.dropna(how='all')
        
        # Check if df[1] is NaN and use df[0] value for a new column 'location'
        df['location'] = np.where(pd.isna(df[df.columns[1]]), df[df.columns[0]], df[df.columns[1]])
        df['location'] = df['location'].astype(str)
                
        # Identify strings containing numbers and replace with NaN
        pattern = r'\d'  # Regular expression to match any digit
        df['location'] = df['location'].where(~df['location'].str.contains(pattern), np.nan)

        # Forward-fill the 'location' column
        df['location'] = df['location'].fillna(method='ffill')
        
        # Create a new column 'types' at rows where df[0] is in uppercase
        df['types'] = np.where(df[df.columns[0]].str.isupper(), df[df.columns[0]], np.nan)
        
        # Forward-fill the 'types' column until a different value is encountered
        df['types'] = df['types'].fillna(method='ffill')

        # Remove rows where 'district_mukim' is equal to 'location'
        df = df[df[df.columns[0]] != df[df.columns[4]]]
                
        # Add a new column 'filename' with the extracted information
        df['source'] = url_file
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
final_result_df = pd.concat(dfs, ignore_index=True)

# Display the final result DataFrame
final_result_df


c:\Users\najmi.ariffin\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,district_mukim,avg_floorsm,rental_2021,rental_2022,location,types,source
0,Taman Maluri,75.0,"1,200 - 1,600","1,200 - 1,600",Mukim Ampang,SINGLE STOREY TERRACE,2022/WP KL
1,Bangsar Park,129.0,"2,300 - 2,500","2,300 - 2,500",Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
2,Cheras Heights Condominium (Bukit Cheras),200.0,NaN,1800,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
3,Lucky Garden,110.0,"2,800 - 3,000","2,800 - 3,000",Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
4,Taman Desa Bakti,84.17,1200,1200,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
...,...,...,...,...,...,...,...
4403,Vivacity Jazz Suites,93.0,"1,500 - 2,000","1,500 - 2,000",Bahagian Kuching,CONDOMINIUM,2022/SARAWAK
4404,The Ochard Residences,145.0,"1,000 - 1,500","1,000 - 1,500",Bahagian Samarahan,CONDOMINIUM,2022/SARAWAK
4405,Waterfront Residence,156.0,"2,000 - 3,000","2,000 - 3,000",Bahagian Sibu,CONDOMINIUM,2022/SARAWAK
4406,Serene Height,150.0,NaN,"950 - 1,300",Bahagian Miri,CONDOMINIUM,2022/SARAWAK


In [11]:
woPPTGSBH = final_result_df

In [12]:
woPPTGSBH

,district_mukim,avg_floorsm,rental_2021,rental_2022,location,types,source
0,Taman Maluri,75.0,"1,200 - 1,600","1,200 - 1,600",Mukim Ampang,SINGLE STOREY TERRACE,2022/WP KL
1,Bangsar Park,129.0,"2,300 - 2,500","2,300 - 2,500",Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
2,Cheras Heights Condominium (Bukit Cheras),200.0,NaN,1800,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
3,Lucky Garden,110.0,"2,800 - 3,000","2,800 - 3,000",Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
4,Taman Desa Bakti,84.17,1200,1200,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
...,...,...,...,...,...,...,...
4403,Vivacity Jazz Suites,93.0,"1,500 - 2,000","1,500 - 2,000",Bahagian Kuching,CONDOMINIUM,2022/SARAWAK
4404,The Ochard Residences,145.0,"1,000 - 1,500","1,000 - 1,500",Bahagian Samarahan,CONDOMINIUM,2022/SARAWAK
4405,Waterfront Residence,156.0,"2,000 - 3,000","2,000 - 3,000",Bahagian Sibu,CONDOMINIUM,2022/SARAWAK
4406,Serene Height,150.0,NaN,"950 - 1,300",Bahagian Miri,CONDOMINIUM,2022/SARAWAK


In [13]:
woPPTGSBH.shape[0]

4408

PPinang

In [14]:
import pandas as pd
from io import BytesIO
import requests
import numpy as np
import xlrd
import openpyxl

# Sample URLs
urls = [
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/PULAU%20PINANG/Jadual%20Harga%20dan%20Sewa%20Pulau%20Pinang%20LPH%202022.xls"
    # "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/TERENGGANU/Jadual%20Harga%20dan%20Sewa%20Terengganu%20LPH%202022.xlsx"
    # "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/SABAH/Jadual%20Harga%20dan%20Sewa%20Sabah%20LPH%202022.xlsx"
 ]

# Create an empty list to store DataFrames
dfs = []

# Specify the sheet names you want to read
target_sheets = ['.3']  # Sheets ending with '.1' or '.3'

# Iterate through each URL
for url in urls:
    # Get the content of the file from the URL
    response = requests.get(url)
    
    # Read sheets that match the criteria
    xls = pd.ExcelFile(BytesIO(response.content))
    sheet_names = [sheet for sheet in xls.sheet_names if sheet.endswith(tuple(target_sheets))]
    
    # Extract relevant information from the URL
    url_parts = url.split('/')
    url_file = f"{url_parts[-3]}/{url_parts[-2]}"
    
    # Iterate through each sheet
    for sheet in sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet, skiprows=5)#,
        df = df.iloc[1:,:4]
        df.columns = ['district_mukim','avg_floorsm','rental_2021','rental_2022']
        
        # Drop rows with all NaN values
        df = df.dropna(how='all')
        
        # Check if df[1] is NaN and use df[0] value for a new column 'location'
        df['location'] = np.where(pd.isna(df[df.columns[1]]), df[df.columns[0]], df[df.columns[1]])
        df['location'] = df['location'].astype(str)
                
        # Identify strings containing numbers and replace with NaN
        pattern = r'\d'  # Regular expression to match any digit
        df['location'] = df['location'].where(~df['location'].str.contains(pattern), np.nan)

        # Forward-fill the 'location' column
        df['location'] = df['location'].fillna(method='ffill')
        
        # Create a new column 'types' at rows where df[0] is in uppercase
        df['types'] = np.where(df[df.columns[0]].str.isupper(), df[df.columns[0]], np.nan)
        
        # Forward-fill the 'types' column until a different value is encountered
        df['types'] = df['types'].fillna(method='ffill')

        # Remove rows where 'district_mukim' is equal to 'location'
        df = df[df[df.columns[0]] != df[df.columns[4]]]
                
        # Add a new column 'filename' with the extracted information
        df['source'] = url_file
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
final_result_df = pd.concat(dfs, ignore_index=True)

# Display the final result DataFrame
woPPonly = final_result_df

In [15]:
woPPonly

,district_mukim,avg_floorsm,rental_2021,rental_2022,location,types,source
0,Green Gardens,167.000000,800 - 850,850,Timur Laut,SINGLE STOREY TERRACE,2022/PULAU%20PINANG
1,Reservoir Garden,130.000000,1100,"1,100 - 1,200",Timur Laut,SINGLE STOREY TERRACE,2022/PULAU%20PINANG
2,Taman Lip Sin,90.000000,900 - 950,"900 - 1,000",Timur Laut,SINGLE STOREY TERRACE,2022/PULAU%20PINANG
3,Prestige III,130.000000,850,900,Barat Daya,SINGLE STOREY TERRACE,2022/PULAU%20PINANG
4,Taman Iping,92.000000,1400,1500,Barat Daya,SINGLE STOREY TERRACE,2022/PULAU%20PINANG
...,...,...,...,...,...,...,...
361,Residensi Sejati,110.095000,"1,200 - 1,800","1,200 - 1,800",Seberang Perai Tengah,CONDOMINIUM,2022/PULAU%20PINANG
362,River Tropics,132.223333,"1,000 - 1,250","900 - 1,400",Seberang Perai Tengah,CONDOMINIUM,2022/PULAU%20PINANG
363,The Prominence,126.852000,"1,000 - 1,500","1,000 - 1,500",Seberang Perai Tengah,CONDOMINIUM,2022/PULAU%20PINANG
364,The Signature,100.980000,1100,"1,100 - 1,200",Seberang Perai Tengah,CONDOMINIUM,2022/PULAU%20PINANG


In [16]:
woPPonly.shape[0]

366

Terengganu

In [17]:
import pandas as pd
from io import BytesIO
import requests
import numpy as np
import xlrd
import openpyxl

# Sample URLs
urls = [
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/TERENGGANU/Jadual%20Harga%20dan%20Sewa%20Terengganu%20LPH%202022.xlsx"
    # "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/SABAH/Jadual%20Harga%20dan%20Sewa%20Sabah%20LPH%202022.xlsx"
 ]

# Create an empty list to store DataFrames
dfs = []

# Specify the sheet names you want to read
target_sheets = ['.3']  # Sheets ending with '.1' or '.3'

# Iterate through each URL
for url in urls:
    # Get the content of the file from the URL
    response = requests.get(url)
    
    # Read sheets that match the criteria
    xls = pd.ExcelFile(BytesIO(response.content))
    sheet_names = [sheet for sheet in xls.sheet_names if sheet.endswith(tuple(target_sheets))]
    
    # Extract relevant information from the URL
    url_parts = url.split('/')
    url_file = f"{url_parts[-3]}/{url_parts[-2]}"
    
    # Iterate through each sheet
    for sheet in sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet, skiprows=5)#,
        df = df.iloc[1:,:4]
        df.columns = ['district_mukim','avg_floorsm','rental_2021','rental_2022']
        
        # Drop rows with all NaN values
        df = df.dropna(how='all')
        
        # Check if df[1] is NaN and use df[0] value for a new column 'location'
        df['location'] = np.where(pd.isna(df[df.columns[1]]), df[df.columns[0]], df[df.columns[1]])
        df['location'] = df['location'].astype(str)
                
        # Identify strings containing numbers and replace with NaN
        pattern = r'\d'  # Regular expression to match any digit
        df['location'] = df['location'].where(~df['location'].str.contains(pattern), np.nan)

        # Forward-fill the 'location' column
        df['location'] = df['location'].fillna(method='ffill')
        
        # Create a new column 'types' at rows where df[0] is in uppercase
        df['types'] = np.where(df[df.columns[0]].str.isupper(), df[df.columns[0]], np.nan)
        
        # Forward-fill the 'types' column until a different value is encountered
        df['types'] = df['types'].fillna(method='ffill')

        # Remove rows where 'district_mukim' is equal to 'location'
        df = df[df[df.columns[0]] != df[df.columns[4]]]
                
        # Add a new column 'filename' with the extracted information
        df['source'] = url_file
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
final_result_df = pd.concat(dfs, ignore_index=True)

# Display the final result DataFrame
woTRGonly = final_result_df

In [18]:
woTRGonly

,district_mukim,avg_floorsm,rental_2021,rental_2022,location,types,source
0,Banggol Peradong,57.0,200 - 300,250 - 300,Kuala Terengganu,SINGLE STOREY LOW - COST TERRACE,2022/TERENGGANU
1,Bukit Kecil,58.0,350 - 450,350 - 500,Kuala Terengganu,SINGLE STOREY LOW - COST TERRACE,2022/TERENGGANU
2,Pekan Manir,62.0,350 - 450,350 - 450,Kuala Terengganu,SINGLE STOREY LOW - COST TERRACE,2022/TERENGGANU
3,Bukit Tunggal,58.0,300 - 450,300 - 450,Kuala Nerus,SINGLE STOREY LOW - COST TERRACE,2022/TERENGGANU
4,Gong Badak,58.0,450 - 500,450 - 500,Kuala Nerus,SINGLE STOREY LOW - COST TERRACE,2022/TERENGGANU
...,...,...,...,...,...,...,...
184,Pangsapuri Permint Harmoni,103.0,"900 - 1,100","900 - 1,100",Kuala Terengganu,APARTMENT,2022/TERENGGANU
185,Pangsapuri Sri Wangi,101.0,850 - 950,850 - 950,Kuala Terengganu,APARTMENT,2022/TERENGGANU
186,Taman Puncak Kemajuan (Blok G),123.0,"800 - 1,000","800 - 1,000",Kuala Terengganu,APARTMENT,2022/TERENGGANU
187,Tok Pelam,113.0,"900 - 1,000","900 - 1,000",Kuala Terengganu,APARTMENT,2022/TERENGGANU


In [19]:
import pandas as pd
from io import BytesIO
import requests
import numpy as np
import xlrd
import openpyxl

# Sample URLs
urls = [
    "https://napic2.jpph.gov.my/storage/app/media//3-penerbitan/pasaran-harta-tanah/laporan-pasaran-harta-tahunan/2022/SABAH/Jadual%20Harga%20dan%20Sewa%20Sabah%20LPH%202022.xlsx"
 ]

# Create an empty list to store DataFrames
dfs = []

# Specify the sheet names you want to read
target_sheets = ['.3']  # Sheets ending with '.1' or '.3'

# Iterate through each URL
for url in urls:
    # Get the content of the file from the URL
    response = requests.get(url)
    
    # Read sheets that match the criteria
    xls = pd.ExcelFile(BytesIO(response.content))
    sheet_names = [sheet for sheet in xls.sheet_names if sheet.endswith(tuple(target_sheets))]
    
    # Extract relevant information from the URL
    url_parts = url.split('/')
    url_file = f"{url_parts[-3]}/{url_parts[-2]}"
    
    # Iterate through each sheet
    for sheet in sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet, skiprows=5)#,
        df = df.iloc[1:,:4]
        df.columns = ['district_mukim','avg_floorsm','rental_2021','rental_2022']
        
        # Drop rows with all NaN values
        df = df.dropna(how='all')
        
        # Check if df[1] is NaN and use df[0] value for a new column 'location'
        df['location'] = np.where(pd.isna(df[df.columns[1]]), df[df.columns[0]], df[df.columns[1]])
        df['location'] = df['location'].astype(str)
                
        # Identify strings containing numbers and replace with NaN
        pattern = r'\d'  # Regular expression to match any digit
        df['location'] = df['location'].where(~df['location'].str.contains(pattern), np.nan)

        # Forward-fill the 'location' column
        df['location'] = df['location'].fillna(method='ffill')
        
        # Create a new column 'types' at rows where df[0] is in uppercase
        df['types'] = np.where(df[df.columns[0]].str.isupper(), df[df.columns[0]], np.nan)
        
        # Forward-fill the 'types' column until a different value is encountered
        df['types'] = df['types'].fillna(method='ffill')

        # Remove rows where 'district_mukim' is equal to 'location'
        df = df[df[df.columns[0]] != df[df.columns[4]]]
                
        # Add a new column 'filename' with the extracted information
        df['source'] = url_file
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
final_result_df = pd.concat(dfs, ignore_index=True)

# Display the final result DataFrame
woSBHonly = final_result_df

In [20]:
woSBHonly

,district_mukim,avg_floorsm,rental_2021,rental_2022,location,types,source
0,Taman Foh Sang,95.0,750,750,Beaufort,SINGLE STOREY TERRACE,2022/SABAH
1,Taman Selagon,78.0,NaN,900,Beaufort,SINGLE STOREY TERRACE,2022/SABAH
2,Taman Sri Panglima,68.0,700,700,Beaufort,SINGLE STOREY TERRACE,2022/SABAH
3,Taman Adika Phase 1,78.0,NaN,900,Keningau,SINGLE STOREY TERRACE,2022/SABAH
4,Taman Cerijaya,80.0,NaN,650,Keningau,SINGLE STOREY TERRACE,2022/SABAH
...,...,...,...,...,...,...,...
570,The Riverside Residence,88.0,NaN,"1,350 - 1,500",Penampang,CONDOMINIUM,2022/SABAH
571,NaN,110.0,"1,850 - 1,900",1850,Penampang,CONDOMINIUM,2022/SABAH
572,Kingfisher Putatan,94.0,"1,000 - 1,300","1,000 - 1,400",Putatan,CONDOMINIUM,2022/SABAH
573,NaN,99.0,"1,050 - 1,200","1,050 - 1,200",Putatan,CONDOMINIUM,2022/SABAH


All

In [31]:
alloc.columns

Index(['district_mukim', 'avg_floorsm', 'rental_2021', 'rental_2022',
       'location', 'types', 'source'],
      dtype='object')

In [34]:
alloc = pd.concat([woPPTGSBH, woPPonly, woTRGonly, woSBHonly], axis=0)
alloc[['rental_2021', 'rental_2022']] = alloc[['rental_2021', 'rental_2022']].replace({',': ''}, regex=True)

alloc.head()

,district_mukim,avg_floorsm,rental_2021,rental_2022,location,types,source
0,Taman Maluri,75.0,1200 - 1600,1200 - 1600,Mukim Ampang,SINGLE STOREY TERRACE,2022/WP KL
1,Bangsar Park,129.0,2300 - 2500,2300 - 2500,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
2,Cheras Heights Condominium (Bukit Cheras),200.0,NaN,1800,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
3,Lucky Garden,110.0,2800 - 3000,2800 - 3000,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
4,Taman Desa Bakti,84.17,1200,1200,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL


In [35]:
alloc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5538 entries, 0 to 574
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   district_mukim  5339 non-null   object
 1   avg_floorsm     5528 non-null   object
 2   rental_2021     4988 non-null   object
 3   rental_2022     5526 non-null   object
 4   location        5538 non-null   object
 5   types           5538 non-null   object
 6   source          5538 non-null   object
dtypes: object(7)
memory usage: 346.1+ KB


In [36]:
alloc.shape[0]

5538

In [37]:
print(woPPTGSBH.shape[0])
print(woPPonly.shape[0])
print(woTRGonly.shape[0])
print(woSBHonly.shape[0])

4408
366
189
575


In [38]:
4408+366+189+575

5538

In [39]:
alloc.head()

,district_mukim,avg_floorsm,rental_2021,rental_2022,location,types,source
0,Taman Maluri,75.0,1200 - 1600,1200 - 1600,Mukim Ampang,SINGLE STOREY TERRACE,2022/WP KL
1,Bangsar Park,129.0,2300 - 2500,2300 - 2500,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
2,Cheras Heights Condominium (Bukit Cheras),200.0,NaN,1800,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
3,Lucky Garden,110.0,2800 - 3000,2800 - 3000,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL
4,Taman Desa Bakti,84.17,1200,1200,Mukim Kuala Lumpur,SINGLE STOREY TERRACE,2022/WP KL


In [40]:
alloc.to_csv('napic_sewa2022.csv', index=False)

In [41]:
alloc.columns

Index(['district_mukim', 'avg_floorsm', 'rental_2021', 'rental_2022',
       'location', 'types', 'source'],
      dtype='object')

In [42]:
alloc.groupby('source').size()

source
2022/JOHOR              809
2022/KEDAH              272
2022/KELANTAN           291
2022/MELAKA             278
2022/NEGERI SEMBILAN    334
2022/PAHANG             427
2022/PERAK              416
2022/PERLIS             198
2022/PULAU%20PINANG     366
2022/SABAH              575
2022/SARAWAK            287
2022/SELANGOR           581
2022/TERENGGANU         189
2022/WP KL              419
2022/WP LABUAN           69
2022/WP PUTRAJAYA        27
dtype: int64

In [57]:
# alloc.query("types == 'FLAT'")
flat_mask = alloc['types'].str.contains('FLAT')
flat_df = alloc[flat_mask]
flat_df

,district_mukim,avg_floorsm,rental_2021,rental_2022,location,types,source
101,Seksyen 1 - 100,NaN,NaN,NaN,LOW COST FLAT,LOW COST FLAT,2022/WP KL
102,Apartment Abdullah Hukum,70.0,1500 - 2100,1500 - 2100,LOW COST FLAT,LOW COST FLAT,2022/WP KL
103,Li Garden,60.0,600 - 850,600 - 850,Mukim Batu,LOW COST FLAT,2022/WP KL
104,Mutiara Fadason,65.0,650 - 800,650 - 800,Mukim Batu,LOW COST FLAT,2022/WP KL
105,Mutiara Magna - Metro Prima,61.0,850 - 900,850 - 900,Mukim Batu,LOW COST FLAT,2022/WP KL
...,...,...,...,...,...,...,...
313,Taman Pasir Putih Phase 2,56.0,550,550,Putatan,LOW - COST FLAT,2022/SABAH
314,Taman Pasir Putih Phase 3B,60.0,750,750,Putatan,LOW - COST FLAT,2022/SABAH
315,LPPB Condominium (Low Cost),47.0,400 - 500,400 - 500,Sandakan,LOW - COST FLAT,2022/SABAH
316,Taman Sejati,56.0,500 - 700,500 - 700,Sandakan,LOW - COST FLAT,2022/SABAH


In [59]:
flat_df.to_csv('flat_df_sewa.csv', index=False)

In [56]:
alloc[alloc['source']=='2022/PERLIS'].groupby('location').size()

location
                       1
Arau                  20
Beseri                17
Jejawi                 8
Kangar               105
Kayang                 2
Kuala Perlis          25
Kurong Anai           10
Kurung Anai            2
Mukim Padang Pauh      1
Padang Besar           1
Padang Siding          1
Titi Tinggi            5
dtype: int64

In [51]:
alloc[alloc['source']=='2022/WP KL'].groupby('location').size()

location
APARTMENT/ CONDOMINIUM     70
DOUBLE STOREY TERRACE       3
LOW COST FLAT               2
Mukim Ampang               10
Mukim Batu                110
Mukim Cheras                2
Mukim Kuala Lumpur         85
Mukim Petaling             71
Mukim Setapak              66
dtype: int64

In [52]:
alloc[alloc['source']=='2022/WP PUTRAJAYA'].groupby('district_mukim').size()

district_mukim
Aura Residence, Presint 8         1
Dwiputra Residence, Presint 15    1
Pangsapuri Putra Damai            1
Pangsapuri Putra Harmoni          1
Park Village, Presint 15          1
Presint 11                        2
Presint 14                        3
Presint 15                        1
Presint 16                        4
Presint 17                        1
Presint 18                        1
Presint 18                        1
Presint 5                         1
Presint 6                         1
Presint 8                         2
Presint 8                         1
Presint 9                         1
Saujana Aster, Presint 11         1
Tamara Residence, Presint 8       1
dtype: int64